Stikprøvestørrelse:

In [31]:
from statsmodels.stats.power import NormalIndPower
import numpy as np

# Parameters
alpha = 0.1            # Significance level
power = 0.8            # Desired power
p0 = 0.5               # Null hypothesis proportion
p1 = 0.65              # Expected proportion

# Calculate effect size
def effect_size(p1, p0):
    return (p1 - p0) / np.sqrt(p0 * (1 - p0))

es = effect_size(p1, p0)

# Compute sample size
analysis = NormalIndPower()
sample_size = analysis.solve_power(effect_size=es, alpha=alpha, power=power, alternative='two-sided')

print(f"Required sample size: {np.ceil(sample_size)}")


Required sample size: 138.0


In [44]:
65/108

0.6018518518518519

Statistisk undersøgelse af forskel på Engelsk og Dansk prompt

In [84]:
# Konfidensinterval for én andel: Menneskelig vurdering
n = 108
forskellig = 65 # indsæt tal for hvor mange, der er forskellige (0) både for menenskelige vurderinger og chattens vurderinger

import statsmodels.stats.proportion as smprop
smprop.proportion_confint(forskellig, n, alpha=0.1) # 90% konfidensinterval


(0.524373115367311, 0.6793305883363927)

In [85]:
# Konfidensinterval for én andel: Chat vurdering
n = 108
forskellig = 72 # indsæt tal for hvor mange, der er forskellige (0) både for menenskelige vurderinger og chattens vurderinger

import statsmodels.stats.proportion as smprop
smprop.proportion_confint(forskellig, n, alpha=0.1) # 90% konfidensinterval

(0.5920545909704196, 0.7412787423629137)

In [65]:
# (Manuel metode)
# compute the standard error
p_hat = 65/108
print(p_hat)
kri_værdi = 1.645
se_p_hat = np.sqrt(p_hat*(1-p_hat)/n)

# compute confidence-interval using normal-approximation
print([p_hat - kri_værdi*se_p_hat, p_hat + kri_værdi*se_p_hat])

0.6018518518518519
[0.5243662206511676, 0.6793374830525362]


Comparing human eval with ChatGPT Eval:

- McNemars Test:

In [81]:
from statsmodels.stats.contingency_tables import mcnemar
import numpy as np

# Confusion table
#              Human Same   Human diff
# Copilot Same       a          b
# Copilot Diff       c          d

menneske = [
    1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
    0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
    1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
    1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
    1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
    1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
    0, 1, 0, 0, 1, 1, 0, 0, 0, 1,
    1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
    1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
    1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
    0, 1, 1, 0, 1, 1, 1, 1
]
copilot = [
    1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
    0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
    0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
    1, 0, 0, 1, 0, 1, 0, 1, 1, 1,
    1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
    1, 1, 1, 1, 1, 1, 1, 1, 0,
    0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
    1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
    1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
    1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
    1, 0, 1, 1, 0, 1, 1, 1, 1
]

a = sum((np.array(menneske) == 1) & (np.array(copilot) == 1))
b = sum((np.array(menneske) == 0) & (np.array(copilot) == 1))
c = sum((np.array(menneske) == 1) & (np.array(copilot) == 0))
d = sum((np.array(menneske) == 0) & (np.array(copilot) == 0))

table = np.array([[a, b],
                  [c, d]])

print(table)

result = mcnemar(table, exact=False, correction=True)
print(f"Statistic: {result.statistic}")
print(f"P-value: {result.pvalue}")

[[57 15]
 [ 7 29]]
Statistic: 2.227272727272727
P-value: 0.13559300126630158


In [83]:
57+15+7+29

108

Multiple hypothesis testing

In [ ]:
# Calculate p-values for each group

# Array for andele af forskellige i hver gruppe

# Array af antal obs i grupper:

z_obs,p_value = smprop.proportions_ztest(count=forskellig, nobs=n, value=0.5, prop_var=0.5)
print(z_obs, p_value)

In [ ]:
from statsmodels.stats.multitest import multipletests

pvals = []

reject, pvals_corrected, _, _ = multipletests(pvals, alpha=0.1, method='fdr_bh')

print("Significant results:", reject)
print("Adjusted p-values:", pvals_corrected)


## Chi^2 test

In [13]:
import scipy.stats as stats
import numpy as np
groups=['Samfund og Histore','Naturvidenskab','Etik og Moral','Erhverv og Økonomi']
table_human=np.array([[8,18,21,17],
            [19,9,6,10]])
table_gpt=np.array([[12,17,25,18],
                    [15,10,2,9]])
chi2, p_val, dof, expected = stats.chi2_contingency(table_gpt, correction=False)
print(p_val)
print(expected)
print(dof)

0.0024848142804233505
[[18. 18. 18. 18.]
 [ 9.  9.  9.  9.]]
3


In [14]:
for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        sub_table = table_gpt[:, [i, j]]
        chi2, p, _, _ = stats.chi2_contingency(sub_table)
        print(f"{groups[i]} vs {groups[j]}: p-value = {p:.4f}")


Samfund og Histore vs Naturvidenskab: p-value = 0.2750
Samfund og Histore vs Etik og Moral: p-value = 0.0004
Samfund og Histore vs Erhverv og Økonomi: p-value = 0.1709
Naturvidenskab vs Etik og Moral: p-value = 0.0219
Naturvidenskab vs Erhverv og Økonomi: p-value = 1.0000
Etik og Moral vs Erhverv og Økonomi: p-value = 0.0426


In [10]:
alpha_bonferroni=0.05/6
alpha_bonferroni

0.008333333333333333